In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from pathlib import Path

import os, sys
sys.path.append(os.path.abspath("../src"))
from data_pipeline.config import PROCESSED_DIR, CLEANED, RAW_DIR


In [4]:
def flatten_cols(df):
    if isinstance(df.columns, pd.MultiIndex):
        if df.columns.get_level_values(1).nunique() == 1:
            df.columns = df.columns.get_level_values(0)
        else:
            df.columns = ["_".join(map(str, c)).strip("_") for c in df.columns]
    return df

In [5]:
# --- Realized Volatility (S&P 500; daily→monthly/quarterly RMS) ---
px = yf.download("^GSPC", start="1950-01-01", interval="1d", progress=False).dropna()
px.index = pd.to_datetime(px.index).tz_localize(None)

logret = np.log(px['Close']).diff()

display(px.head(3))
display(logret.head(3))

/var/folders/2p/b19f37nj6b98f824990xs99m0000gn/T/ipykernel_537/3475955613.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download("^GSPC", start="1950-01-01", interval="1d", progress=False).dropna()


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
1950-01-03,16.66,16.66,16.66,16.66,1260000
1950-01-04,16.85,16.85,16.85,16.85,1890000
1950-01-05,16.93,16.93,16.93,16.93,2550000


Ticker,^GSPC
Date,
1950-01-03,NaN
1950-01-04,0.011340
1950-01-05,0.004737


In [6]:
spy = yf.download("SPY", start="1950-01-01", interval="1d", progress=False).dropna()
spy.index = pd.to_datetime(spy.index).tz_localize(None)
display(spy.head(3))

/var/folders/2p/b19f37nj6b98f824990xs99m0000gn/T/ipykernel_537/3372152514.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  spy = yf.download("SPY", start="1950-01-01", interval="1d", progress=False).dropna()


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
1993-01-29,24.313038,24.330330,24.209284,24.330330,1003200
1993-02-01,24.485968,24.485968,24.330336,24.330336,480500
1993-02-02,24.537838,24.555130,24.416792,24.468669,201300


In [7]:
spy = flatten_cols(spy)
print(spy.columns)
display(spy.head(3))

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')


Price,Close,High,Low,Open,Volume
Date,,,,,
1993-01-29,24.313038,24.330330,24.209284,24.330330,1003200
1993-02-01,24.485968,24.485968,24.330336,24.330336,480500
1993-02-02,24.537838,24.555130,24.416792,24.468669,201300


In [8]:
px = flatten_cols(px)
print(px.columns)
display(px.head(3))

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')


Price,Close,High,Low,Open,Volume
Date,,,,,
1950-01-03,16.66,16.66,16.66,16.66,1260000
1950-01-04,16.85,16.85,16.85,16.85,1890000
1950-01-05,16.93,16.93,16.93,16.93,2550000


In [9]:
# --- Realized volatility over next 21 days (annualized) ---
window = 21
rv_21d = np.sqrt(
    logret.rolling(window, min_periods=window)
           .apply(lambda x: (x**2).sum(), raw=True)
           .shift(-window) * (252 / window)
)

rv_21d = rv_21d.rename(columns={"^GSPC": 'rv_21d'}) 
rv_21d = rv_21d.dropna()

rv_21d.head()

Ticker,rv_21d
Date,
1950-01-03,0.106477
1950-01-04,0.105440
1950-01-05,0.104849
1950-01-06,0.104523
1950-01-09,0.104101


In [10]:
spy = spy.rename(columns={
    'Open': 'open_spy', 'High': 'high_spy', 'Low': 'low_spy', 
    'Close': 'close_spy', 'Adj Close': 'adjclose_spy'
})
spx = px.rename(columns={
    'Open': 'open_gspc', 'High': 'high_gspc', 'Low': 'low_gspc', 
    'Close': 'close_gspc', 'Adj Close': 'adjclose_gspc'
})

spy = spy.drop(columns='Volume', errors='ignore')
spx  = spx.drop(columns='Volume', errors='ignore')

display(spy.head(3))
display(spx.head(3))

Price,close_spy,high_spy,low_spy,open_spy
Date,,,,
1993-01-29,24.313038,24.330330,24.209284,24.330330
1993-02-01,24.485968,24.485968,24.330336,24.330336
1993-02-02,24.537838,24.555130,24.416792,24.468669


Price,close_gspc,high_gspc,low_gspc,open_gspc
Date,,,,
1950-01-03,16.66,16.66,16.66,16.66
1950-01-04,16.85,16.85,16.85,16.85
1950-01-05,16.93,16.93,16.93,16.93


In [11]:
spx = spx.join(rv_21d, how='inner')
display(spx.head(3))

spy = spy.join(rv_21d, how='inner')
display(spy.head(3))

,close_gspc,high_gspc,low_gspc,open_gspc,rv_21d
Date,,,,,
1950-01-03,16.66,16.66,16.66,16.66,0.106477
1950-01-04,16.85,16.85,16.85,16.85,0.105440
1950-01-05,16.93,16.93,16.93,16.93,0.104849


,close_spy,high_spy,low_spy,open_spy,rv_21d
Date,,,,,
1993-01-29,24.313038,24.330330,24.209284,24.330330,0.126903
1993-02-01,24.485968,24.485968,24.330336,24.330336,0.123896
1993-02-02,24.537838,24.555130,24.416792,24.468669,0.124781


In [12]:
spx = spx.reset_index().rename(columns={'Date': 'date'})
spy = spy.reset_index().rename(columns={'Date': 'date'})

In [13]:
display(spx.head(3))
display(spy.head(3))

,date,close_gspc,high_gspc,low_gspc,open_gspc,rv_21d
0,1950-01-03,16.66,16.66,16.66,16.66,0.106477
1,1950-01-04,16.85,16.85,16.85,16.85,0.105440
2,1950-01-05,16.93,16.93,16.93,16.93,0.104849


,date,close_spy,high_spy,low_spy,open_spy,rv_21d
0,1993-01-29,24.313038,24.330330,24.209284,24.330330,0.126903
1,1993-02-01,24.485968,24.485968,24.330336,24.330336,0.123896
2,1993-02-02,24.537838,24.555130,24.416792,24.468669,0.124781


In [14]:
spy.to_csv(RAW_DIR / "SPY_data.csv", index=False)
spx.to_csv(RAW_DIR / "GSPC_data.csv", index=False)